<a href="https://colab.research.google.com/github/Nathan-oc/Projetos-Python/blob/main/Baralhos_Anki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução

O objetivo deste projeto é criar um programa em Python capaz de gerar um baralho de flashcards contendo frases em inglês e suas respectivas traduções para o português. Antes de explicarmos como o programa funciona, vamos entender o que são flashcards e como eles podem ser úteis no processo de aprendizagem.

Um flashcard é um cartão com informações úteis que podem ser usadas para estudo ou memorização. Na maioria das vezes, os flashcards contêm perguntas em um lado e respostas no outro, mas eles também podem ser usados para memorizar outras informações, como fórmulas matemáticas, vocabulário de idiomas, datas históricas e muito mais.

O principal objetivo do uso de flashcards é ajudar a memorizar informações importantes. Eles são frequentemente usados por estudantes que querem aprender um novo idioma, por exemplo. Ao criar flashcards com palavras em inglês e suas traduções para o português, é possível treinar a memória visual e a associação entre as palavras em ambos os idiomas.

O programa que estamos desenvolvendo é capaz de buscar frases em inglês e suas traduções para o português em um dicionário online e adicioná-las em um baralho de flashcards. Isso significa que o usuário não precisa gastar tempo procurando frases adequadas e traduções precisas - o programa faz isso automaticamente.

Para buscar as frases em inglês e suas traduções para o português, nosso programa utiliza o site wordreference.com, que é um dicionário online confiável e muito utilizado por estudantes de línguas estrangeiras. O wordreference.com possui uma grande base de dados de frases em inglês, com suas traduções correspondentes em português, o que torna o processo de busca e coleta de informações mais fácil e eficiente. Isso significa que o programa pode buscar as informações necessárias com rapidez e precisão, e assim criar os flashcards com as frases corretas e suas traduções correspondentes.

Nos próximos passos, explicaremos como o programa funciona e como ele pode ser usado para criar baralhos de flashcards personalizados com frases em inglês e português.

# Importação de Bibliotecas

Para este projeto em particular, vamos precisar das bibliotecas `selenium`, `genanki` e `pandas`.

A biblioteca `selenium` é usada para automatizar a navegação na web, permitindo a interação com sites da mesma forma que um usuário faria. Já a biblioteca `genanki` é usada para criar um arquivo Anki com os flashcards que serão gerados pelo programa. Por fim, a biblioteca `pandas` é usada para manipulação de dados, permitindo que o programa leia e armazene as palavras em inglês em um formato estruturado.

In [ ]:
from selenium.webdriver.common.by import By
from selenium import webdriver
import genanki
import pandas as pd

In [ ]:
#sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options = chrome_options)

# Definição de funções

Nesta seção, vamos explicar cada uma das funções desenvolvidas neste projeto em Python para a criação de um baralho dos flashcards.



* `trad_wr`: Essa função recebe como entrada uma palavra em inglês e utiliza a biblioteca Selenium para realizar uma busca no site WordReference. Em seguida, ela coleta as frases em inglês e suas respectivas traduções para o português e retorna duas listas: uma contendo as frases em inglês e outra contendo as traduções em português correspondentes.

* `add_cards`: Essa função recebe como entrada um objeto de deck do Genanki, uma lista contendo as frases em inglês e outra lista contendo as respectivas traduções em português, geradas pela função `trad_wr()`. Para cada frase em inglês e sua respectiva tradução, a função gera um card e adiciona ao deck.

* `lista_palavras`: Essa função é responsável por criar um loop que recebe palavras em inglês do usuário até que ele digite um valor para parar o loop (por exemplo, 0). Essa função retorna uma lista com as palavras inseridas pelo usuário.

* `baralho_ENPT`: Essa função é responsável por criar um objeto de deck do Genanki e para cada palavra da lista gerada pela função `lista_palavras()`, ela chama as funções `trad_wr()` e `add_cards()` para gerar e adicionar cards ao deck. Por fim, ela gera um arquivo em formato .apkg contendo o baralho criado.

* `main()`: Essa função é a função principal do projeto. Ela chama a função `lista_palavras()` para gerar a lista de palavras em inglês a serem adicionadas ao baralho, e em seguida chama a função `baralho_ENPT()` para criar o baralho de flashcards.

In [ ]:
def trad_wr(palavra):
  
  # Entra na url a partir da palavra pesquisada
  pal = str(palavra)
  url = 'https://www.wordreference.com/enpt/' + pal
  driver.get(url)

  # Gera objetos que contém as frases
  frases = driver.find_elements(By.XPATH, '//td[@class="FrEx"]')
  traducao = driver.find_elements(By.XPATH, '//td[@class="ToEx"]')
    
  frases_lista = []
  traducao_lista = []
  
  f, t = len(frases), len(traducao)

  # Lista para adicionar as strings das frases em inglês e suas traduções
  strings = []
  # Lista binária, onde 0: frase inglês e 1: traduçao para o português
  lista = []
  # Lista que guarda a coordenada y da string na página web
  coord_y = []

  # Para cada frase do inglês adiciona suas informações às listas já deifinidas
  for i in range(f):
    strings.append(frases[i].text)
    lista.append(0)
    coord_y.append(frases[i].location['y'])

  # Para cada tradução adiciona as informações às listas
  for i in range(t):  
    strings.append(traducao[i].text)
    lista.append(1)
    coord_y.append(traducao[i].location['y'])

  # Gera um dataframe que contém as informações das frases
  data = {'string': strings, 'lista': lista, 'y': coord_y}
  df = pd.DataFrame(data)
  
  # Reordena as linhas do dataframe baseado na localização vertical da string na página web
  df = df.sort_values(['y'], ascending = True)
  df = df.reset_index(drop=True)

  # Adiciona às listas de frases e traduções apenas o par de strings que possuem a sequência frase->tradução
  for i in df.loc[df['lista'].diff()==1].index:
    frases_lista.append(df.loc[i - 1, 'string'])
    traducao_lista.append(df.loc[i, 'string'])
 
  return frases_lista, traducao_lista


def add_cards(deck, frases_lista, traducao_lista):
    
  # Gera um modelo de flashcard
  modelo = genanki.Model(
  1236326257,
  'Simples',
  fields=[
    {'name': 'Question'},
    {'name': 'Answer'},
  ],
  templates=[
    {
      'name': 'Card 1',
      'qfmt': '{{Question}}',
      'afmt': '{{FrontSide}}<hr id="answer">{{Answer}}',
    },
  ])


  # Adiciona um card ao baralho para cada par (frase,tradução)
  for i in range(len(frases_lista)):
    
    frase = str(frases_lista[i])
    traducao = str(traducao_lista[i])
    
    card = genanki.Note(
    model=modelo,
    fields=[frase, traducao])
    
    deck.add_note(card)



def baralho_ENPT(lista_palavras):

  # Gera um baralho
  baralho = genanki.Deck(
  1462467949,
  'EN-PT')  

  # Adiciona todas as cartas geradas ao deck, baseado na lista de palavras 
  for palavra in lista_palavras:
    frases_lista, traducao_lista = trad_wr(palavra)
    add_cards(baralho, frases_lista, traducao_lista)

  genanki.Package(baralho).write_to_file('EN-PT.apkg')


def lista_palavras():

  lista_palavras = []
  palavra = str(input("Palavra (Digite 0 para pausar): "))
  
  while palavra != '0':
    lista_palavras.append(palavra)
    palavra = str(input("Palavra (Digite 0 para pausar): "))

  return lista_palavras


def main():

  lista = lista_palavras()
  baralho_ENPT(lista)

# Execução Função Main

In [ ]:
main()

Palavra (Digite 0 para pausar):  data
Palavra (Digite 0 para pausar):  science
Palavra (Digite 0 para pausar):  analysis
Palavra (Digite 0 para pausar):  0
